<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240127-%E5%BE%97%E5%88%B03class%E6%A0%B7%E6%9C%AC%E7%9A%842022NDVI%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.21-8.23).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [3]:
import os
import pandas as pd
import math

In [4]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/sample_3class")

In [5]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [6]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [7]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-03-21'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-08-23').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [8]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI']


In [9]:
image_agrgt_clip = image_agrgt.clip(roi)

In [10]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id','landcover'],scale=10,tileScale=8)

In [11]:
cols = ['id', 'landcover', '0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI']
inds = range(samples_datesets.size().getInfo())

In [12]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,landcover,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,...,22_NDVI,23_NDVI,24_NDVI,25_NDVI,26_NDVI,27_NDVI,28_NDVI,29_NDVI,30_NDVI,31_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(34),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
id,52.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.000000,61.000000,62.000000,63.000000,...,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000,50.000000,51.000000
landcover,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
0_NDVI,0.530435,0.695009,0.717196,0.593773,0.530206,0.613082,0.558747,0.616357,0.555715,0.523565,...,0.416804,0.413983,0.365708,0.482627,0.554445,0.390709,0.345079,0.454216,0.531847,0.285952
1_NDVI,0.533381,0.698135,0.723304,0.598946,0.539148,0.617713,0.561365,0.630506,0.563241,0.527213,...,0.419661,0.415580,0.370273,0.491095,0.565220,0.398322,0.350870,0.460258,0.540327,0.288258
2_NDVI,0.536378,0.700565,0.728807,0.603939,0.548227,0.622425,0.564335,0.644626,0.570921,0.531077,...,0.422823,0.417500,0.375128,0.499700,0.576100,0.406012,0.357192,0.465996,0.548386,0.290693


In [14]:
df.to_csv('sample_NDVI_2022_247.csv',encoding='utf-8')